In [ ]:
# -*- coding: utf-8 -*-
"""
DIC-9345 - Projet 2: Traduction Automatique Neuronale (TAN) - EN->RU
Version pour entraînement complet sur opus_books (plus de données, plus d'époques).
Utilise do_eval=False pour éviter TypeError. Batch size 8.
"""

# @title 1. Installation des bibliothèques nécessaires
# Installe les bibliothèques Hugging Face (transformers, datasets), SacreBLEU et Accelerate.
!pip install transformers[torch] datasets sacrebleu accelerate evaluate -q

print("Installation terminée.")

# @title 2. Importations et Configuration Initiale
import os
# Silence XLA/TensorFlow CUDA warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
# Désactive Weights & Biases (si non utilisé)
os.environ["WANDB_DISABLED"] = "true"

import torch
import numpy as np
# Utilisation de 'evaluate' au lieu de 'load_metric' pour les métriques
from datasets import load_dataset
import evaluate # Nouvelle façon de charger les métriques
from transformers import (
    MarianTokenizer, # Utilisation explicite
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

# Configuration pour Anglais -> Russe
MODEL_CHECKPOINT = "Helsinki-NLP/opus-mt-en-ru" # Modèle pré-entraîné Anglais -> Russe
SOURCE_LANG = "en"
TARGET_LANG = "ru"
# Utilisation d'opus_books
DATASET_NAME = "opus_books"
DATASET_CONFIG = "en-ru"

# Limites pour l'exemple (COMMENTÉES POUR UTILISER TOUTES LES DONNÉES)
# MAX_TRAIN_SAMPLES = 10000
# MAX_VAL_SAMPLES = 1000
# MAX_TEST_SAMPLES = 1000
MAX_INPUT_LENGTH = 128
MAX_TARGET_LENGTH = 128

# Vérification explicite du device GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation du périphérique : {device}")
if torch.cuda.is_available():
    # Afficher le nom du GPU pour confirmer (devrait être T4 si sélectionné)
    print(f"Nom du GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Aucun GPU détecté, utilisation du CPU.")


# @title 3. Chargement et Prétraitement des Données

# Charger le jeu de données (opus_books en-ru)
try:
    raw_datasets_full = load_dataset(DATASET_NAME, DATASET_CONFIG)
    print(f"Dataset {DATASET_NAME} ({DATASET_CONFIG}) chargé.")
    # Inspecter la structure, opus_books a juste 'train'
    print(raw_datasets_full)
    # opus_books n'a qu'un split 'train', nous devons le diviser manuellement
    # Utiliser une plus grande partie pour l'entraînement maintenant
    train_test_split = raw_datasets_full['train'].train_test_split(test_size=0.05, seed=42) # 5% pour test
    train_val_split = train_test_split['train'].train_test_split(test_size=0.05, seed=42) # 5% du reste pour validation (soit ~4.75% du total)

    raw_datasets = {
        'train': train_val_split['train'],
        'validation': train_val_split['test'],
        'test': train_test_split['test']
    }
    print("Dataset divisé en train/validation/test.")
    # Affichage des tailles réelles qui seront utilisées
    print(f"Tailles réelles des splits - Train: {len(raw_datasets['train'])}, Validation: {len(raw_datasets['validation'])}, Test: {len(raw_datasets['test'])}")


except Exception as e:
    print(f"Erreur lors du chargement ou de la division du dataset {DATASET_NAME} ({DATASET_CONFIG}): {e}")
    raise e # Arrêter si le dataset ne peut être chargé

# Charger le tokenizer correspondant au modèle pré-entraîné
tokenizer = MarianTokenizer.from_pretrained(MODEL_CHECKPOINT)
print(f"Tokenizer chargé pour {MODEL_CHECKPOINT}")

# Fonction de prétraitement pour tokeniser les paires de phrases
def preprocess_function(examples):
    # Accède aux données textuelles correctement, que ce soit via 'translation' ou directement
    if "translation" in examples: # Pour WMT-style datasets
      inputs = [ex[SOURCE_LANG] for ex in examples["translation"]]
      targets = [ex[TARGET_LANG] for ex in examples["translation"]]
    else: # Pour opus_books-style datasets (qui a 'id' et 'translation' comme colonnes)
      # On doit accéder aux langues DANS la colonne 'translation'
      inputs = [ex[SOURCE_LANG] for ex in examples["translation"]]
      targets = [ex[TARGET_LANG] for ex in examples["translation"]]

    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True, padding=False) # Padding sera géré par DataCollator
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=MAX_TARGET_LENGTH, truncation=True, padding=False)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Appliquer le prétraitement aux datasets
# Ajout de num_proc pour potentiellement accélérer cette étape sur CPU
num_cpus = os.cpu_count()
print(f"Utilisation de {num_cpus} coeurs pour le prétraitement.")
tokenized_datasets = {}
for split, dataset in raw_datasets.items():
     cols_to_remove = ['id', 'translation']
     tokenized_datasets[split] = dataset.map(
         preprocess_function,
         batched=True,
         remove_columns=cols_to_remove,
         num_proc=num_cpus, # Utiliser plusieurs coeurs CPU
         desc=f"Tokenizing {split} split..."
     )

print("Prétraitement terminé.")
print("Structure après tokenisation (exemple train):", tokenized_datasets["train"])


# Utiliser les datasets complets (après split)
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"] # Gardé pour référence
test_dataset_tokenized = tokenized_datasets["test"]
test_dataset_raw = raw_datasets["test"] # Garder les refs non tokenisées

# Les lignes .select() pour MAX_..._SAMPLES sont commentées ou supprimées
# if MAX_TRAIN_SAMPLES and MAX_TRAIN_SAMPLES < len(train_dataset):
#     train_dataset = train_dataset.shuffle(seed=42).select(range(MAX_TRAIN_SAMPLES))
# if MAX_VAL_SAMPLES and MAX_VAL_SAMPLES < len(eval_dataset):
#      eval_dataset = eval_dataset.shuffle(seed=42).select(range(MAX_VAL_SAMPLES))
# if MAX_TEST_SAMPLES and MAX_TEST_SAMPLES < len(test_dataset_tokenized):
#     indices = list(range(len(raw_datasets['test'])))
#     np.random.seed(42)
#     np.random.shuffle(indices)
#     selected_indices = indices[:MAX_TEST_SAMPLES]
#     test_dataset_tokenized = tokenized_datasets["test"].select(selected_indices)
#     test_dataset_raw = raw_datasets["test"].select(selected_indices)


print(f"Taille du jeu d'entraînement utilisé: {len(train_dataset)} exemples")
print(f"Taille du jeu d'évaluation utilisé (référence): {len(eval_dataset)} exemples")
print(f"Taille du jeu de test utilisé: {len(test_dataset_tokenized)} exemples (tokenisé)")
print(f"Taille du jeu de test utilisé: {len(test_dataset_raw)} exemples (raw)")


# @title 4. Chargement du Modèle et Configuration de l'Entraînement

# Charger le modèle pré-entraîné Seq2Seq
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
# Rétablissement de l'envoi explicite au device détecté (devrait être cuda)
model.to(device)
print(f"Modèle {MODEL_CHECKPOINT} chargé et envoyé sur {device}.")

# Nom du run pour le suivi
run_name = f"{MODEL_CHECKPOINT.split('/')[-1]}-finetuned-{SOURCE_LANG}-to-{TARGET_LANG}-full-5e" # Entraînement complet, 5 époques

# Arguments d'entraînement - Pour entraînement complet
training_args = Seq2SeqTrainingArguments(
    output_dir=run_name,
    do_train=True,
    do_eval=False,              # PAS d'évaluation pendant l'entraînement
    logging_strategy="steps",
    logging_steps=200,          # Logger la perte tous les 200 pas
    save_strategy="epoch",      # Sauvegarder à chaque époque
    learning_rate=2e-5,
    per_device_train_batch_size=8, # Taille de batch raisonnable pour T4
    per_device_eval_batch_size=8,  # Utilisé seulement par predict()
    weight_decay=0.01,
    save_total_limit=2,          # Garder les 2 derniers checkpoints + le final
    num_train_epochs=5,          # Augmentation à 5 époques
    predict_with_generate=True,
    fp16=torch.cuda.is_available(), # Utiliser la précision mixte si GPU dispo
    push_to_hub=False,
    generation_max_length=MAX_TARGET_LENGTH,
    # report_to="wandb",
)

# Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Métriques d'évaluation (SacreBLEU et chrF) - Utilisées par predict()
sacrebleu_metric = evaluate.load("sacrebleu")
chrf_metric = evaluate.load("chrf")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

# compute_metrics sera appelé par predict() à la fin
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    bleu_result = sacrebleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    # Calcul chrF standard
    chrf_result = chrf_metric.compute(predictions=decoded_preds, references=decoded_labels)

    result = {"bleu": bleu_result["score"], "chrf": chrf_result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

# Initialiser le Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    # eval_dataset=eval_dataset, # Pas nécessaire si do_eval=False
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics, # Passé ici, sera utilisé par predict()
)

print(f"Configuration de l'entraînement terminée ({run_name}, prêt pour GPU, do_eval=False).")


# @title 5. Lancement de l'Entraînement (Fine-tuning)
# ATTENTION: Ceci prendra BEAUCOUP plus de temps.
print(f"Début de l'entraînement ({run_name}) sur GPU...")
try:
    train_result = trainer.train()
    # Pas de "meilleur" modèle à charger, on sauvegarde le dernier état
    trainer.save_model() # Sauvegarde le modèle final après N époques
    metrics = train_result.metrics
    metrics["train_samples"] = len(train_dataset)
    trainer.log_metrics("train", metrics) # Log des métriques d'entraînement
    trainer.save_metrics("train", metrics) # Sauvegarde des métriques d'entraînement
    trainer.save_state()
    print("Entraînement terminé et modèle final sauvegardé.")
except Exception as e:
    print(f"Une erreur est survenue pendant l'entraînement : {e}")
    if "CUDA out of memory" in str(e):
        print("Erreur 'CUDA out of memory'. Essayez de réduire 'per_device_train_batch_size'.")
    print("L'entraînement a été interrompu.")


# @title 6. Évaluation sur le Jeu de Test
print("Début de l'évaluation finale sur le jeu de test avec le modèle final...")
# Le modèle dans trainer est le dernier après N époques
model.eval()

# Utiliser trainer.predict() pour l'évaluation finale sur le jeu de test
# compute_metrics sera appelé ici
predict_results = trainer.predict(test_dataset_tokenized, metric_key_prefix="test")

metrics = predict_results.metrics
metrics["test_samples"] = len(test_dataset_raw) # Utilise la taille réelle après sélection

print(f"----- Résultats de l'évaluation finale sur le jeu de test ({run_name}) -----")
print(f"Score SacreBLEU: {metrics.get('test_bleu', 'N/A'):.4f}")
print(f"Score chrF: {metrics.get('test_chrf', 'N/A'):.4f}")

# Recalculons pour avoir tous les détails de SacreBLEU si nécessaire
if predict_results.predictions is not None:
    preds = predict_results.predictions
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    references = [ex['translation'][TARGET_LANG] for ex in test_dataset_raw]
    cleaned_preds, cleaned_labels = postprocess_text(decoded_preds, references)

    final_bleu_metric = evaluate.load("sacrebleu")
    test_bleu_results_detailed = final_bleu_metric.compute(predictions=cleaned_preds, references=cleaned_labels)

    print(f"(Recalculé) Score SacreBLEU: {test_bleu_results_detailed['score']:.4f}")
    if 'precisions' in test_bleu_results_detailed:
        print(f"Précisions BLEU (1-4 grams): { [round(p, 4) for p in test_bleu_results_detailed['precisions']] }")
    print(f"Ratio de brièveté (BP): {test_bleu_results_detailed.get('bp', 'N/A'):.4f}")
    print(f"Longueur moyenne des prédictions: {np.mean([len(p.split()) for p in cleaned_preds]):.2f} mots")
    print(f"Longueur moyenne des références: {np.mean([len(l[0].split()) for l in cleaned_labels]):.2f} mots")

    output_prediction_file = os.path.join(run_name, "test_predictions_ru.txt")
    output_reference_file = os.path.join(run_name, "test_references_ru.txt")
    with open(output_prediction_file, "w", encoding="utf-8") as writer:
        writer.write("\n".join(cleaned_preds))
    with open(output_reference_file, "w", encoding="utf-8") as writer:
        writer.write("\n".join([ref[0] for ref in cleaned_labels]))
    print(f"Prédictions sauvegardées dans: {output_prediction_file}")
    print(f"Références sauvegardées dans: {output_reference_file}")

    # On ajoute les précisions BLEU au dictionnaire AVANT de sauvegarder
    metrics["test_bleu_precisions"] = test_bleu_results_detailed.get('precisions')
    metrics["test_bp"] = test_bleu_results_detailed.get('bp')

# Commentaire de log_metrics pour éviter TypeError
# trainer.log_metrics("test", metrics)
# Sauvegarder les métriques (y compris la liste des précisions) dans le fichier JSON
trainer.save_metrics("test", metrics)

print("Évaluation finale terminée.")


# @title 7. Exemple d'Inférence (Traduction d'une phrase EN->RU)
# Le modèle dans trainer est le dernier après N époques
print("\nExemple d'inférence avec le modèle final...")
sentence_en = "Machine translation is fascinating."
print(f"Phrase source ({SOURCE_LANG}): {sentence_en}")

# Tokenizer sur CPU, envoi des tenseurs au device du modèle
inputs = tokenizer(sentence_en, return_tensors="pt").to(device) # Envoyer les inputs au device

with torch.no_grad():
    # generate s'exécute sur le device du modèle
    outputs = model.generate(**inputs, max_length=MAX_TARGET_LENGTH, num_beams=4, early_stopping=True)

# outputs sont sur le device, décoder directement
translation_ru = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Traduction ({TARGET_LANG}): {translation_ru}")

# Autre exemple
sentence_en_2 = "This model was trained on the full dataset for more epochs."
print(f"\nPhrase source ({SOURCE_LANG}): {sentence_en_2}")
inputs_2 = tokenizer(sentence_en_2, return_tensors="pt").to(device) # Envoyer les inputs au device
with torch.no_grad():
    outputs_2 = model.generate(**inputs_2, max_length=MAX_TARGET_LENGTH, num_beams=4, early_stopping=True)
translation_ru_2 = tokenizer.decode(outputs_2[0], skip_special_tokens=True)
print(f"Traduction ({TARGET_LANG}): {translation_ru_2}")


print(f"\nScript ({run_name}) terminé pour EN->RU (GPU).")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17496 [00:00<?, ? examples/s]

Dataset opus_books (en-ru) chargé.
DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 17496
    })
})
Dataset divisé en train/validation/test.
Tailles réelles des splits - Train: 15789, Validation: 832, Test: 875


...

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Modèle Helsinki-NLP/opus-mt-en-ru chargé et envoyé sur cpu.


/tmp/ipython-input-461475538.py:223: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Configuration de l'entraînement terminée (opus-mt-en-ru-finetuned-en-to-ru-full-5e, prêt pour GPU, do_eval=False).
Début de l'entraînement (opus-mt-en-ru-finetuned-en-to-ru-full-5e) sur GPU...


Step,Training Loss
200,2.019000
400,1.977500
600,1.862100
800,1.810500
1000,1.817200
1200,1.815400
1400,1.765000
